Given the output we get from the "basic_neural_network" notebook, the accuracy we get when testing our neural network on new data is poor even when we train it with large data set (24000 sequences). <br/>
This is of course related to the fact that we only make use of (part of) the DNA sequence itself and no extra information is incorporated in the input. <br/>
This notebook trys to incorporate the Transcription Factor Binding Site information into the current data set and  explores different ways to accomplish this.

### To Do
1. Merge the information of TF binding sites in various ways and prepare it properly into the desired format.
2. Design the Neural Network architecture (without implementation)
3. Hopefully, test some architecture designs and see how it improves(?) the result accuracy

### Incorporate the TF Information and Prepare the Data